In [2]:
import csv
import folium
import openrouteservice
import pandas as pd
from geopy.geocoders import Nominatim
from branca.element import Figure
from openrouteservice import convert


#座標取得
geo_list = []
geo = pd.read_csv("geo_point.csv")
geolocator = Nominatim(user_agent="user-id")
for chimei in range(len(geo)):
    location = geolocator.geocode(geo.iloc[chimei,0]+",江東区")
    geo_list.append([location.address.split(",")[0],location.latitude, location.longitude,geo.iloc[chimei,1]])
df_list = pd.DataFrame(geo_list, columns=["地名","latitude","longitude","stay_time"])
df_list.to_csv("geo_loc_pos.csv", index=False, encoding="utf_8")


line1 = pd.read_csv("geo_loc_pos.csv")
color_list = ["red","green","purple","orange","darkred","lightred","beige","darkblue","darkgreen","cadetblue","darkpurple","white","pink","lightblue","lightgreen","gray","black","lightgray","blue"]
points_a = []
for i in range(len(line1)):
    points_a.append([line1.iloc[i,1],line1.iloc[i,2]])
    

key = "5b3ce3597851110001cf624804ffaeec7cd246038d01eb4d3a32f633"
client = openrouteservice.Client(key=key)

loc_place = []
for chimei in range(len(points_a)-1):
    p1 = points_a[chimei]
    p2 = points_a[chimei+1]
    p1r = tuple(reversed(p1))
    p2r = tuple(reversed(p2))
    mean_lat = (p1[0] + p2[0]) / 2
    mean_long = (p1[1] + p2[1]) / 2

    # 経路計算 (Directions V2)
    routedict = client.directions((p1r, p2r),profile="foot-walking")
    geom = routedict["routes"][0]["geometry"]
    decoded = convert.decode_polyline(geom)
    for i in range(len(decoded["coordinates"])):
        loc_place.append(decoded["coordinates"][i])
        

datalist = []

with open("geo_loc_pos.csv", encoding="utf_8") as f:
    reader = csv.reader(f)
    line = [row for row in reader]

for i in range(len(line)-2):
    p1 = float(line[i+1][1]), float(line[i+1][2])
    for j in range(i,len(line)-2):
        p2 = float(line[j+2][1]), float(line[j+2][2])
        p1r = tuple(reversed(p1))
        p2r = tuple(reversed(p2))
        mean_lat = (p1[0] + p2[0]) / 2
        mean_long = (p1[1] + p2[1]) / 2

        # 経路計算 (Directions V2)
        routedict = client.directions((p1r, p2r),profile="foot-walking")
        geom = routedict["routes"][0]["geometry"]
        decoded = convert.decode_polyline(geom)
        datalist.append([i,j+1,float(routedict["routes"][0]["summary"]["duration"])])
df_list = pd.DataFrame(datalist, columns=["出発地点","行先","移動時間[s]"])
df_list.to_csv("geo_loc_loc_time.csv", index=False, encoding="utf_8")